In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import omegaconf
from shell.utils.experiment_utils import *
from shell.fleet.utils.fleet_utils import *
from shell.utils.metric import *
import matplotlib.pyplot as plt
from shell.fleet.network import TopologyGenerator
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
from shell.fleet.fleet import Agent, Fleet
from shell.fleet.data.data_utilize import *
from shell.fleet.data.recv import *

from sklearn.manifold import TSNE
from torchvision.utils import make_grid
from shell.fleet.data.data_utilize import *
import logging
from sklearn.metrics import f1_score
import os
from shell.fleet.data.recv_utils import *
from tqdm import tqdm
import argparse
from functools import partial
from torchvision.utils import make_grid
from shell.utils.oodloss import OODSeparationLoss
from pythresh.thresholds.dsn import DSN
from pythresh.thresholds.aucp import AUCP
from pythresh.thresholds.boot import BOOT
from pythresh.thresholds.zscore import ZSCORE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import defaultdict
from experiments.run import handle_combine_modes

logging.basicConfig(level=logging.INFO)

In [3]:
experiment_folder = "."
experiment_name = "combine_modes_results/modmod"
# experiment_name = "combine_modes_results/debug_modmod"
dataset = "mnist"
algo = "modular"

use_contrastive = False
num_trains_per_class = 64 if dataset != "cifar100" else 256
seed = 0
parallel = False
task_id = 5



agent_id = 5

In [4]:
def modify_cfg(net_cfg, agent_cfg, train_cfg, fleet_additional_cfg, cfg):
    cfg = handle_combine_modes(cfg)
    return  net_cfg, agent_cfg, train_cfg, fleet_additional_cfg, cfg

In [5]:
save_dir = get_save_dir(experiment_folder, experiment_name,
                        dataset, algo, num_trains_per_class, use_contrastive, seed)

fleet = setup_fleet(save_dir, task_id=task_id-1, parallel=False, modify_cfg=modify_cfg)
# fleet = setup_fleet(save_dir, task_id=task_id, parallel=False, modify_cfg=modify_cfg)
fleet.change_save_dir('debug_combine_modes_results')

INFO:root:Class sequence: [5 0 4 3 4 6 4 3 8 0 1 0 5 9 6 4 9 0 3 0]
INFO:root:task 0 :(128, 1, 28, 28) (100, 1, 28, 28) (1872, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28) (100, 1, 28, 28) (1992, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28) (100, 1, 28, 28) (1940, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28) (100, 1, 28, 28) (1992, 1, 28, 28)
INFO:root:task 4 :(128, 1, 28, 28) (100, 1, 28, 28) (1954, 1, 28, 28)


{'train': {'component_update_freq': 100, 'num_epochs': 100, 'init_component_update_freq': 100, 'init_num_epochs': 100, 'save_freq': 10}, 'dataset': {'dataset_name': 'mnist', 'num_tasks': 10, 'num_classes_per_task': 2, 'with_replacement': True, 'num_trains_per_class': 64, 'num_vals_per_class': 50, 'remap_labels': True}, 'net': {'name': 'mlp', 'depth': 4, 'layer_size': 64, 'dropout': 0.5, 'no_sparse_basis': True}, 'sharing_strategy': {'name': 'combine_modes', 'communicator': 'modmod', 'sync_base': True, 'comm_freq': 'None', 'num_coms_per_round': 'None', 'pre_or_post_comm': 'None'}, 'seed': 0, 'algo': 'modular', 'job_name': 'mnist_modular_numtrain_64', 'num_agents': 8, 'root_save_dir': 'combine_modes_results/modmod', 'parallel': True, 'num_init_tasks': 4, 'overwrite': False, 'topology': 'fully_connected', 'edge_drop_prob': 0.0, 'agent': {'save_dir': '${root_save_dir}/${job_name}/${dataset.dataset_name}/${algo}/seed_${seed}', 'batch_size': 64, 'memory_size': 32, 'improvement_threshold': 0.

INFO:root:task 5 :(128, 1, 28, 28) (100, 1, 28, 28) (2115, 1, 28, 28)
INFO:root:task 6 :(128, 1, 28, 28) (100, 1, 28, 28) (1901, 1, 28, 28)
INFO:root:task 7 :(128, 1, 28, 28) (100, 1, 28, 28) (1940, 1, 28, 28)
INFO:root:task 8 :(128, 1, 28, 28) (100, 1, 28, 28) (1989, 1, 28, 28)
INFO:root:task 9 :(128, 1, 28, 28) (100, 1, 28, 28) (1990, 1, 28, 28)
INFO:root:Class sequence: [4 7 5 7 6 0 3 0 5 0 3 6 2 7 6 7 6 1 0 5]
INFO:root:task 0 :(128, 1, 28, 28) (100, 1, 28, 28) (2010, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28) (100, 1, 28, 28) (1920, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28) (100, 1, 28, 28) (1938, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28) (100, 1, 28, 28) (1990, 1, 28, 28)
INFO:root:task 4 :(128, 1, 28, 28) (100, 1, 28, 28) (1872, 1, 28, 28)
INFO:root:task 5 :(128, 1, 28, 28) (100, 1, 28, 28) (1968, 1, 28, 28)
INFO:root:task 6 :(128, 1, 28, 28) (100, 1, 28, 28) (2060, 1, 28, 28)
INFO:root:task 7 :(128, 1, 28, 28) (100, 1, 28, 28) (1986, 1, 28, 28)
INFO:root:task 8 :(128

i_size 28
num_classes 2
net_cfg {'name': 'mlp', 'depth': 4, 'layer_size': 64, 'dropout': 0.5, 'no_sparse_basis': True, 'i_size': 28, 'num_classes': 2, 'num_tasks': 10, 'num_init_tasks': 4, 'use_contrastive': False}
<class 'shell.learners.er_dynamic.CompositionalDynamicER'>


INFO:root:task 6 :(128, 1, 28, 28) (100, 1, 28, 28) (1940, 1, 28, 28)
INFO:root:task 7 :(128, 1, 28, 28) (100, 1, 28, 28) (2010, 1, 28, 28)
INFO:root:task 8 :(128, 1, 28, 28) (100, 1, 28, 28) (1989, 1, 28, 28)
INFO:root:task 9 :(128, 1, 28, 28) (100, 1, 28, 28) (2145, 1, 28, 28)
INFO:root:Created fleet with 8 agents
INFO:root:Adding neighbors...
INFO:root:Fleet initialized


Setting seed 69420000
Setting seed 0
Setting seed 1000
Setting seed 2000
Setting seed 3000
Setting seed 4000
Setting seed 5000
Setting seed 6000
Setting seed 7000
Setting seed 0
Setting seed 2000
Setting seed 4000
Setting seed 6000
Setting seed 8000
Setting seed 10000
Setting seed 12000
Setting seed 14000


In [6]:
agent = fleet.agents[agent_id]

In [7]:
agent.eval_test(task_id)

{0: 0.9910358565737052,
 1: 0.9534764826175869,
 2: 0.9183467741935484,
 3: 0.9593858345715701,
 4: 0.7036664992466097,
 5: 0.49461312438785504}

In [8]:
agent.net.structure[task_id]

Parameter containing:
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], device='cuda:0', requires_grad=True)

In [9]:
agent.net.decoder[task_id].bias

Parameter containing:
tensor([0.0039, 0.0020], device='cuda:0', requires_grad=True)

In [10]:
len(agent.net.components)

5

In [11]:
agent.net.components[-1].bias

Parameter containing:
tensor([-1.3570e-01,  1.4027e-01, -1.1317e-01, -1.9371e-01,  8.7128e-02,
         2.7102e-02,  6.2681e-02, -1.2516e-01, -6.3624e-02,  1.6261e-01,
         8.7935e-02,  3.2667e-02,  6.2471e-02, -4.2881e-02,  2.1601e-04,
         6.8511e-02, -6.1917e-02, -4.4565e-02,  1.2035e-01,  9.4327e-02,
        -3.3057e-02, -1.5331e-01,  3.8375e-02,  5.9515e-02,  1.2773e-01,
        -1.4533e-01,  9.9504e-02, -1.4493e-01,  4.0874e-02,  1.3322e-02,
        -3.6597e-02, -4.0041e-03, -9.5925e-02,  2.1736e-01,  6.9136e-02,
         8.7045e-02,  8.7189e-02,  4.2614e-03,  2.7338e-02,  6.4767e-02,
        -9.2970e-02, -3.6907e-02, -6.7393e-03, -6.3056e-02, -6.9952e-02,
        -1.0675e-01, -1.1564e-01,  1.1179e-01, -3.2081e-02, -6.4876e-02,
         2.4955e-02, -6.6618e-02,  1.1987e-01, -1.7567e-02, -6.3354e-02,
         3.3180e-02,  1.7006e-02,  4.0631e-02, -1.1317e-01, -1.1865e-02,
         3.0548e-02, -1.4667e-01,  1.4212e-01, -2.2762e-02], device='cuda:0',
       requires_grad=Tru

In [12]:
agent.train_kwargs

{'component_update_freq': 100,
 'num_epochs': 100,
 'init_component_update_freq': 100,
 'init_num_epochs': 100,
 'save_freq': 10}

In [13]:
len(agent.net.components)

5

In [14]:
print(fleet.comm_freq)
print(fleet.num_coms_per_round)
# print(agent.communicator)

{'modmod': 100}
{'modmod': 2}


In [15]:
 if "debug" in experiment_name:
    fleet.communicate(task_id=task_id, end_epoch=None, comm_freq=None, num_epochs=None,
                      final=False, strategy=None)
else:
     fleet.communicate(task_id=task_id, end_epoch=None, comm_freq=None, num_epochs=None,
                      final=False, strategy='modmod')

In [16]:
agent.eval_test(task_id)

{0: 0.9910358565737052,
 1: 0.9534764826175869,
 2: 0.9183467741935484,
 3: 0.9593858345715701,
 4: 0.7036664992466097,
 5: 0.5166503428011753}

In [17]:
module_list = agent.train_kwargs['module_list']
agent.net.add_tmp_modules(task_id, len(module_list))
agent.net.receive_modules(task_id, module_list)

In [18]:
agent.eval_test(task_id)

{0: 0.9910358565737052,
 1: 0.9534764826175869,
 2: 0.9183467741935484,
 3: 0.9593858345715701,
 4: 0.7036664992466097,
 5: 0.544564152791381}

In [19]:
agent.net.structure[task_id]

Parameter containing:
tensor([[1.4840, 0.4587, 0.5361, 0.5790],
        [0.4887, 1.4869, 1.5785, 1.1688],
        [0.5110, 0.4690, 0.5012, 1.2600],
        [0.8829, 0.5690, 0.6640, 0.9629],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [1.2591, 0.8578, 0.7942, 0.9756]], device='cuda:0', requires_grad=True)

In [20]:
agent.net.decoder[task_id].bias

Parameter containing:
tensor([ 0.0000, -0.2600], device='cuda:0', requires_grad=True)

In [21]:
agent.net.components[-1].bias

Parameter containing:
tensor([-0.1194, -0.1430, -0.0294,  0.0593,  0.0244, -0.0044,  0.1163, -0.0692,
        -0.0310,  0.2710, -0.0194,  0.0104, -0.0545,  0.0956, -0.1445,  0.0016,
        -0.0152,  0.0275,  0.0909,  0.0609,  0.2795,  0.2023, -0.1384,  0.2812,
        -0.0087,  0.1847, -0.0212,  0.0128, -0.0474, -0.1065,  0.0686, -0.0626,
         0.2702,  0.1696,  0.0433, -0.0011,  0.1918,  0.0345,  0.0078,  0.1957,
        -0.0520, -0.0256, -0.0270, -0.0610,  0.0260, -0.0547,  0.1945, -0.0516,
        -0.0487,  0.0400,  0.0752, -0.0348,  0.0382,  0.0798,  0.0699,  0.0447,
         0.0101,  0.0686, -0.1059,  0.0615,  0.0714,  0.0135, -0.1370, -0.1210],
       device='cuda:0', requires_grad=True)

In [23]:
agent.net.random_linear_projection.bias

Parameter containing:
tensor([ 0.0299,  0.0179, -0.0248, -0.0030,  0.0317,  0.0197,  0.0019, -0.0297,
         0.0142,  0.0171,  0.0028,  0.0060,  0.0273,  0.0238,  0.0337,  0.0083,
         0.0228,  0.0204, -0.0061, -0.0154, -0.0331, -0.0043, -0.0158, -0.0150,
         0.0270,  0.0300,  0.0274,  0.0340, -0.0186,  0.0352,  0.0137, -0.0048,
        -0.0341, -0.0071, -0.0105, -0.0140,  0.0161, -0.0187,  0.0221, -0.0319,
         0.0210,  0.0006,  0.0009,  0.0161, -0.0069, -0.0037,  0.0314, -0.0171,
        -0.0197,  0.0077, -0.0148, -0.0228,  0.0247,  0.0115, -0.0002, -0.0278,
        -0.0060, -0.0124, -0.0181, -0.0329,  0.0004, -0.0152,  0.0060, -0.0029],
       device='cuda:0')

In [25]:
fleet.agents[4].net.random_linear_projection.bias

Parameter containing:
tensor([ 0.0299,  0.0179, -0.0248, -0.0030,  0.0317,  0.0197,  0.0019, -0.0297,
         0.0142,  0.0171,  0.0028,  0.0060,  0.0273,  0.0238,  0.0337,  0.0083,
         0.0228,  0.0204, -0.0061, -0.0154, -0.0331, -0.0043, -0.0158, -0.0150,
         0.0270,  0.0300,  0.0274,  0.0340, -0.0186,  0.0352,  0.0137, -0.0048,
        -0.0341, -0.0071, -0.0105, -0.0140,  0.0161, -0.0187,  0.0221, -0.0319,
         0.0210,  0.0006,  0.0009,  0.0161, -0.0069, -0.0037,  0.0314, -0.0171,
        -0.0197,  0.0077, -0.0148, -0.0228,  0.0247,  0.0115, -0.0002, -0.0278,
        -0.0060, -0.0124, -0.0181, -0.0329,  0.0004, -0.0152,  0.0060, -0.0029],
       device='cuda:0')